In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

def co2_emissions_by_region(dict_scenario, folder_path, file_path, run_name,output_folder, called = False):
    if called == True:
        dataframe = pd.read_csv(file_path + 'VAR_FOut_' + run_name + '.csv', sep = ',')
        unique_countries = dataframe['4'].unique()
        num_countries = len(unique_countries)
        num_cols = 3  # Number of columns
        num_rows = -(-num_countries // num_cols)  # Ceiling division to calculate number of rows


        fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 3 * num_rows))

        # Flatten the axes array if only one row of subplots
        if num_rows == 1:
            axes = axes.ravel()
        else:
            axes = [item for sublist in axes for item in sublist]

        for ax, country in zip(axes, unique_countries):
            
            # Filter DataFrame for the current country
            tot_co2_res = pd.DataFrame()
            for filename in dict_scenario.items():
                var_Fout = pd.read_csv(f"{folder_path}/{filename[1]}/VAR_FOut_{filename[1]}.csv", sep = ',')
                var_Fout_country = var_Fout[var_Fout['4'] == country]
                tot_co21 = var_Fout_country[var_Fout_country['1'] == 'TOTCO2']
                scen_name = filename[0]
                tot_co2 = tot_co21.rename(columns={'8': scen_name})
                tot_co2 = tot_co2.reset_index(drop=True)
                tot_co2 = tot_co2.set_index(tot_co2['3'])
                # print(tot_co2_world)
                tot_co2_res = pd.concat([tot_co2_res,tot_co2[scen_name]/10**3], axis = 1)
            # print(tot_co2_res)
            # Plot with the discrete color map
            for i, scen in enumerate(tot_co2_res.columns):
                colors = ['skyblue','orange','green','red']
                # plt.fill_between(tot_co2_res.index, tot_co2_res[str(scen)], color=colors[i], alpha=0.4, label= str(scen))
                ax.plot(tot_co2_res.index, tot_co2_res[str(scen)], color=colors[::-1][i], linewidth=2,label= str(scen))
            
            # Set labels and title
            ax.legend()
            ax.set_xlabel('Year')
            ax.set_ylabel('Gt CO2')
            ax.set_title(f'Region {country}')
            
            ax.grid(False)
        # Adjust layout
        plt.tight_layout()


        plt.savefig(output_folder + 'CO2_by_region.pdf'
                    , format ='pdf'
                    ,  bbox_inches='tight')
    else:
        pass